In [ ]:
# default_exp att_stats

# AttStats

> Class representing attribute statistics.


Attribute statistics consist of the average confidence for the attribute and the average confidence for each of it's values.

**Example:**

"@attribute outlook {sunny, overcast, rainy}
<br> @attribute temperature {hot, mild, cool}
<br>
<br> @data
<br> sunny[0.5];rainy[0.3];overcast[0.2],hot
<br> sunny[0.1];rainy[0.4];overcast[0.5],mild
<br> sunny[0.7];rainy[0.1];overcast[0.2],cool"

Statistics for "outlook" attribute are as follows:
 * average confidence: ( max(0.5, 0.3, 0.2) + max(0.1, 0.4, 0.5) + max(0.7, 0.1, 0.2) ) / 3 = 0.57
 * statistics: [(0.5 + 0.1 + 0.7) / 3, (0.3 + 0.4 + 0.1) / 3, (0.2 + 0.5 + 0.2) / 3] = [0.43, 0.27, 0.3]

In [ ]:
# export
from typing import List

from pyuid3.value import Value
from pyuid3.attribute import Attribute
# from pyuid3.data import Data   # may cause problems

In [ ]:
# export
class AttStats:
    def __init__(self, statistics: Dict[str,Value], avg_confidence: float):
        self.statistics = statistics
        self.avg_confidence = avg_confidence

    @staticmethod
    def calculate_statistics(att: Attribute, data: 'Data') -> 'AttStats':    # TODO: rename to get_stats
        conf_sum = {}
        avg_conf = 0

        if not data.get_instances():
            return AttStats(conf_sum, avg_conf)

        instances = data.get_instances()
        att_name=att.get_name()
        for instance in instances:
            r = instance.get_reading_for_attribute(att_name)
            values = r.get_values()
            for v in values:
                if v.get_name() in conf_sum.keys():
                    old = conf_sum[v.get_name()]
                    conf_sum[v.get_name()] = Value(v.get_name(), old.get_confidence() + v.get_confidence())
                else:
                    conf_sum[v.get_name()] = v#Value(v.get_name(), old.get_confidence() + v.get_confidence())
            
            avg_conf += r.get_most_probable().get_confidence()

        size = len(data)
        avg_conf /= size
        stats = {}
        for stat_v in conf_sum.values():
            stats[stat_v.get_name()]=(Value(stat_v.get_name(), stat_v.get_confidence()/size))
        return AttStats(stats, avg_conf)




    def get_statistics(self) -> List[Value]:   # TODO: rename to get_statistics
        return list(self.statistics.values())

    def get_avg_confidence(self) -> float:
        return self.avg_confidence

    def get_stat_for_value(self, value_name: str) -> float:
        if value_name in self.statistics.keys():
            return self.statistics[value_name].get_confidence()
        else:
            return 0

    def get_most_probable(self) -> Value:
        statistics = list(self.statistics.values())
        confidence = [value.get_confidence() for value in statistics]
        highest_conf = max(confidence)
        index = confidence.index(highest_conf)
        return statistics[index]

    def __str__(self) -> str:
        result = '{'
        for value in self.statistics.values():
            result += str(value) + ','
        result = result[:-1]  # delete the last coma ','
        result += '}'
        return result


## Examples

In [ ]:
import math
from pyuid3.data import Data

data = Data.parse_uarff("../resources/weather.nominal.uncertain.arff")
attribute = Attribute('outlook', {'sunny', 'rainy', 'overcast'})
att_stats = AttStats.get_statistics(attribute, data)
assert att_stats.get_avg_confidence() == 0.9428571428571428

vals = att_stats.het_statistics()
print(vals[0])
print(vals[1])
print(vals[2])

rainy[0.39]
overcast[0.24]
sunny[0.37]
